## Library Imports

In [1]:
from time import time
notebook_start_time = time()

In [2]:
import re
import torch
from torch import nn
from torchvision import models

## Constants and Utilities

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def breaker(num=50, char="*") -> None:
    print("\n" + num*char + "\n")


def head(x, no_of_ele=5) -> None:
    print(x[:no_of_ele])

## Model

In [4]:
class Model(nn.Module):
    def __init__(self, model_name=None, pretrained=False):
        super(Model, self).__init__()

        if re.match(r"^resnet$", model_name, re.IGNORECASE):
            self.features = models.resnet50(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.features.add_module("Flatten", nn.Flatten())

            in_features = self.features[-3][2].bn3.num_features


        elif re.match(r"^vgg$", model_name, re.IGNORECASE):
            self.features = models.vgg16_bn(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-2])
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(2, 2)))
            self.features.add_module("Flatten", nn.Flatten())

            in_features = self.features[-3][41].num_features * 2 * 2


        elif re.match(r"^mobilenet$", model_name, re.IGNORECASE):
            self.features = models.mobilenet_v2(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
            self.features.add_module("Flatten", nn.Flatten())

            in_features = self.features[-3][-1][1].num_features


        elif re.match(r"^densenet$", model_name, re.IGNORECASE):
            self.features = models.densenet169(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
            self.features.add_module("Flatten", nn.Flatten())

            in_features = self.features[0].norm5.num_features
            
    def freeze(self):
        for params in self.parameters():
                params.requires_grad = False
    
    def forward(self, x):
        return self.features(x)

## Save Pretrained Weights

In [5]:
model_names = ["mobilenet", "densenet", "resnet", "vgg"]

for model_name in model_names:
    model = Model(model_name=model_name, pretrained=True).to(DEVICE)
    torch.save(model.state_dict(), "./{}_state.pt".format(model_name))

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [6]:
breaker()
print("Notebook Run Time : {:.2f} minutes".format((time()-notebook_start_time)/60))
breaker()


**************************************************

Notebook Run Time : 0.65 minutes

**************************************************

